# 📈 Notebook 3: Sistema de Seguimiento de Precios

En este notebook vamos a:
1. Actualizar precios de productos guardados
2. Ver el histórico de cambios de precio
3. Detectar cambios significativos
4. Crear alertas de precio
5. Automatizar el monitoreo

---

## 📚 Setup

In [ ]:
import sys
sys.path.append('../src')

from scraper import MercadoLibreScraper
from database import PriceDatabase
from analyzer import PriceAnalyzer
from utils import (
    format_price, 
    print_product_summary, 
    get_price_change_emoji,
    calculate_percentage_change,
    create_price_alert
)
import pandas as pd
from datetime import datetime, timedelta
import time

# Inicializar
scraper = MercadoLibreScraper()
db = PriceDatabase("../data/prices.db")

print("✓ Módulos cargados")
print(f"📅 Fecha actual: {datetime.now().strftime('%d/%m/%Y %H:%M')}")

## 1️⃣ Ver Productos Monitoreados

Primero veamos qué productos tenemos en seguimiento.

In [9]:
# Obtener todos los productos
products = db.get_all_products()

print(f"📦 Productos en seguimiento: {len(products)}\n")

if products:
    df = pd.DataFrame(products)
    
    # Formatear para visualización
    display_df = df[['title', 'min_price', 'max_price', 'price_count', 'first_seen']].copy()
    
    display_df['min_price'] = display_df['min_price'].apply(lambda x: format_price(x) if x else 'N/A')
    display_df['max_price'] = display_df['max_price'].apply(lambda x: format_price(x) if x else 'N/A')
    
    display_df.columns = ['Título', 'Precio Mín.', 'Precio Máx.', 'Registros', 'Desde']
    
    display(display_df)
else:
    print("⚠️ No hay productos en seguimiento.")
    print("Ve al notebook 02_scraping_basics.ipynb para agregar productos.")

📦 Productos en seguimiento: 1



,Título,Precio Mín.,Precio Máx.,Registros,Desde
0,Sin título,$22.000,$1.732.099,6,2025-12-11 15:24:01


## 2️⃣ Actualizar Precios

Vamos a actualizar los precios de los productos que ya tenemos guardados.

### 2.1 Función para actualizar un producto

In [10]:
def update_product_price(product_data):
    """
    Actualiza el precio de un producto buscándolo nuevamente
    """
    product_id = product_data['id']
    title = product_data['title']
    
    print(f"🔄 Actualizando: {title[:50]}...")
    
    # Buscar el producto por su título
    search_results = scraper.search_products(title, limit=5)
    
    # Intentar encontrar el mismo producto por ID
    found = None
    for result in search_results:
        if result['id'] == product_id:
            found = result
            break
    
    if found:
        # Guardar nuevo precio
        db.save_price(found)
        
        # Obtener histórico para comparar
        history = db.get_price_history(product_id)
        if len(history) >= 2:
            old_price = history[-2]['price']
            new_price = history[-1]['price']
            change = calculate_percentage_change(old_price, new_price)
            emoji = get_price_change_emoji(change)
            
            print(f"  ✓ Actualizado: {format_price(old_price)} → {format_price(new_price)} {emoji}")
        else:
            print(f"  ✓ Precio guardado: {format_price(found['price'])}")
        
        return True
    else:
        print(f"  ✗ No se encontró el producto")
        return False

### 2.2 Actualizar todos los productos

**Nota:** Este proceso puede tardar unos minutos dependiendo de cuántos productos tengas.

In [11]:
if products:
    print("🔄 Iniciando actualización de precios...\n")
    print("="*70)
    
    updated = 0
    failed = 0
    
    for product in products:
        success = update_product_price(product)
        
        if success:
            updated += 1
        else:
            failed += 1
        
        # Delay para no sobrecargar el servidor
        time.sleep(2)
    
    print("\n" + "="*70)
    print(f"\n✓ Actualización completada")
    print(f"   Exitosas: {updated}")
    print(f"   Fallidas: {failed}")
else:
    print("⚠️ No hay productos para actualizar")

🔄 Iniciando actualización de precios...

🔄 Actualizando: Sin título...
Buscando: Sin título
URL: https://listado.mercadolibre.com.ar/Sin-título
✓ Producto encontrado: Sin título...
✓ Producto encontrado: Sin título...
✓ Producto encontrado: Sin título...
✓ Producto encontrado: Sin título...
✓ Producto encontrado: Sin título...
  ✓ Actualizado: $22.000 → $28.900 📈🔴 ¡Gran subida!


✓ Actualización completada
   Exitosas: 1
   Fallidas: 0


## 3️⃣ Analizar Histórico de un Producto

Selecciona un producto y ve su evolución de precio.

In [12]:
if products:
    # Seleccionar el primer producto como ejemplo
    product_to_analyze = products[0]  # 👈 Cambia el índice [0] para ver otros productos
    
    print(f"📊 Analizando: {product_to_analyze['title']}\n")
    print("="*70)
    
    # Obtener histórico
    history = db.get_price_history(product_to_analyze['id'])
    
    if history:
        print(f"\n📈 Histórico de precios ({len(history)} registros):\n")
        
        df_history = pd.DataFrame(history)
        df_history['scraped_at'] = pd.to_datetime(df_history['scraped_at'])
        
        # Mostrar primeros y últimos registros
        display_df = df_history[['scraped_at', 'price', 'seller']].copy()
        display_df['price'] = display_df['price'].apply(format_price)
        display_df.columns = ['Fecha', 'Precio', 'Vendedor']
        
        print("Primeros registros:")
        display(display_df.head())
        
        if len(history) > 5:
            print("\nÚltimos registros:")
            display(display_df.tail())
        
        # Estadísticas
        analyzer = PriceAnalyzer(history)
        stats = analyzer.get_statistics()
        
        print("\n💰 Estadísticas:")
        print("="*50)
        print(f"Precio actual:     {format_price(stats['precio_actual'])}")
        print(f"Precio mínimo:     {format_price(stats['precio_minimo'])}")
        print(f"Precio máximo:     {format_price(stats['precio_maximo'])}")
        print(f"Precio promedio:   {format_price(stats['precio_promedio'])}")
        print(f"Variación:         {stats['variacion_porcentual']:.2f}%")
        print("="*50)
    else:
        print("⚠️ No hay histórico para este producto")

📊 Analizando: Sin título


📈 Histórico de precios (7 registros):

Primeros registros:


,Fecha,Precio,Vendedor
0,2025-12-11 12:23:55.649957,$299.999,Desconocido
1,2025-12-11 12:26:18.813257,$1.354.195,Desconocido
2,2025-12-11 12:26:18.813667,$1.732.099,Desconocido
3,2025-12-11 12:26:18.813826,$948.370,Desconocido
4,2025-12-11 12:31:26.705453,$28.900,Desconocido



Últimos registros:


,Fecha,Precio,Vendedor
2,2025-12-11 12:26:18.813667,$1.732.099,Desconocido
3,2025-12-11 12:26:18.813826,$948.370,Desconocido
4,2025-12-11 12:31:26.705453,$28.900,Desconocido
5,2025-12-11 12:32:04.912088,$22.000,Desconocido
6,2025-12-11 12:32:21.832275,$28.900,Desconocido



💰 Estadísticas:
Precio actual:     $28.900
Precio mínimo:     $22.000
Precio máximo:     $1.732.099
Precio promedio:   $630.638
Variación:         -90.37%


## 4️⃣ Detectar Cambios Significativos

Veamos qué productos tuvieron cambios importantes de precio.

In [13]:
# Detectar cambios (umbral de 5%)
threshold = 5.0  # 👈 Porcentaje mínimo de cambio

print(f"🔍 Buscando cambios de precio ≥ {threshold}%...\n")

changes = db.get_price_changes(threshold=threshold)

if changes:
    print(f"✓ Se encontraron {len(changes)} productos con cambios significativos:\n")
    print("="*70)
    
    for change in changes:
        emoji = get_price_change_emoji(change['change_percent'])
        
        print(f"\n📦 {change['title'][:55]}")
        print(f"   Anterior: {format_price(change['previous_price'])}")
        print(f"   Actual:   {format_price(change['current_price'])}")
        print(f"   Cambio:   {change['change_percent']:.2f}% {emoji}")
    
    print("\n" + "="*70)
else:
    print(f"No se encontraron cambios significativos (>{threshold}%)")
    print("Esto es normal si acabas de empezar a rastrear productos.")

🔍 Buscando cambios de precio ≥ 5.0%...

✓ Se encontraron 1 productos con cambios significativos:


📦 Sin título
   Anterior: $22.000
   Actual:   $28.900
   Cambio:   31.36% 📈🔴 ¡Gran subida!



## 5️⃣ Sistema de Alertas de Precio

Configura alertas para productos específicos.

In [14]:
if products:
    # Configurar alertas para productos
    price_alerts = [
        {
            'product_index': 0,  # 👈 Índice del producto en tu lista
            'target_price': 400000  # 👈 Precio objetivo
        },
        # Puedes agregar más alertas aquí
    ]
    
    print("🔔 Verificando alertas de precio...\n")
    print("="*70)
    
    for alert_config in price_alerts:
        idx = alert_config['product_index']
        target = alert_config['target_price']
        
        if 0 <= idx < len(products):
            product = products[idx]
            history = db.get_price_history(product['id'])
            
            if history:
                current_price = history[-1]['price']
                title = product['title'][:50]
                
                alert_message = create_price_alert(current_price, target, title)
                print(alert_message)
                print()
    
    print("="*70)

🔔 Verificando alertas de precio...


🔔 ¡ALERTA DE PRECIO!
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📦 Producto: Sin título
💰 Precio actual: $28.900
🎯 Tu objetivo: $400.000
✅ El producto está dentro de tu rango de precio!
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        



## 6️⃣ Mejores Ofertas del Momento

Encuentra los productos que están en su precio más bajo.

In [15]:
if products:
    print("🎯 Buscando las mejores ofertas...\n")
    print("="*70)
    
    best_deals = []
    
    for product in products:
        history = db.get_price_history(product['id'])
        
        if len(history) >= 2:
            analyzer = PriceAnalyzer(history)
            recommendation = analyzer.detect_best_time_to_buy()
            
            best_deals.append({
                'title': product['title'],
                'score': recommendation['score'],
                'current_price': recommendation['current_price'],
                'min_price': recommendation['min_price'],
                'recommendation': recommendation['recommendation']
            })
    
    # Ordenar por score (mejores primero)
    best_deals.sort(key=lambda x: x['score'], reverse=True)
    
    if best_deals:
        print("\n🏆 TOP OFERTAS RECOMENDADAS:\n")
        
        for i, deal in enumerate(best_deals[:5], 1):
            stars = "⭐" * deal['score']
            
            print(f"{i}. {deal['title'][:55]}")
            print(f"   {stars} ({deal['score']}/5)")
            print(f"   💰 Precio actual: {format_price(deal['current_price'])}")
            print(f"   💚 Precio mínimo: {format_price(deal['min_price'])}")
            print(f"   📊 {deal['recommendation']}")
            print()
    else:
        print("Necesitas más datos históricos para generar recomendaciones.")
    
    print("="*70)

🎯 Buscando las mejores ofertas...


🏆 TOP OFERTAS RECOMENDADAS:

1. Sin título
   ⭐⭐⭐⭐ (4/5)
   💰 Precio actual: $28.900
   💚 Precio mínimo: $22.000
   📊 Buen momento para comprar. Precio por debajo del promedio



## 7️⃣ Script de Monitoreo Automático

Este script te permitirá programar actualizaciones automáticas.

In [16]:
# Código para crear un script de monitoreo

monitoring_script = '''
#!/usr/bin/env python3
"""
Script de Monitoreo Automático
Ejecuta este script diariamente para actualizar precios
"""

import sys
sys.path.append('src')

from scraper import MercadoLibreScraper
from database import PriceDatabase
from datetime import datetime
import time

def monitor_prices():
    print(f"🤖 Monitoreo automático - {datetime.now()}")
    print("="*70)
    
    scraper = MercadoLibreScraper()
    db = PriceDatabase("data/prices.db")
    
    products = db.get_all_products()
    
    print(f"Actualizando {len(products)} productos...\n")
    
    for product in products:
        print(f"Actualizando: {product['title'][:50]}...")
        
        results = scraper.search_products(product['title'], limit=5)
        
        found = None
        for result in results:
            if result['id'] == product['id']:
                found = result
                break
        
        if found:
            db.save_price(found)
            print(f"  ✓ Actualizado")
        else:
            print(f"  ✗ No encontrado")
        
        time.sleep(2)
    
    print("\n✓ Monitoreo completado")

if __name__ == "__main__":
    monitor_prices()
'''

# Guardar el script
with open('../monitor.py', 'w', encoding='utf-8') as f:
    f.write(monitoring_script)

print("✓ Script de monitoreo creado: monitor.py")
print("\nPara usarlo:")
print("  python monitor.py")
print("\nPara programarlo diariamente (Linux/Mac):")
print("  crontab -e")
print("  Agregar: 0 9 * * * cd /ruta/al/proyecto && python monitor.py")

✓ Script de monitoreo creado: monitor.py

Para usarlo:
  python monitor.py

Para programarlo diariamente (Linux/Mac):
  crontab -e
  Agregar: 0 9 * * * cd /ruta/al/proyecto && python monitor.py


## 🎯 Resumen

En este notebook aprendiste a:

- ✅ Actualizar precios de productos guardados
- ✅ Ver y analizar el histórico de precios
- ✅ Detectar cambios significativos
- ✅ Configurar alertas de precio
- ✅ Encontrar las mejores ofertas
- ✅ Crear un script de monitoreo automático

---

**¡Continúa con el último notebook!** 👉 `04_data_analysis.ipynb`

Allí crearás visualizaciones avanzadas y análisis detallados de los datos.